# Automatic Map Initialization
The main goal of this is to compute the relative pose between 2 frames to triangulate an initial set of map pts. 

The first step is to find initial correspondances. This can be done with ORB and brute-force matching them. 

In [2]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

PATH_TO_IMG1 = "./parallax_1.jpg"
PATH_TO_IMG2 = "./parallax_2.jpg"

In [ ]:
img1 = cv2.imread(PATH_TO_IMG1, cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(PATH_TO_IMG2, cv2.IMREAD_GRAYSCALE)

The next step is to compute a homography and a fundamental matrix on the matches. Technically this can be done in parallel but for demonstration purposes this will be skipped. 

The third step is to select a good model. Ideally one that the fundamental matrix has a good result on and the homography on a subset is ideal. 

The fourth step is to perform motion and structure from 